# Обработка данных Scopus для Khazar University

Этот notebook извлекает новые статьи из экспорта Scopus, которых еще нет в базе United Scopus and WoS.

**Алгоритм:**
1. Загрузить исходный файл Scopus и целевой файл United
2. Найти статьи, которых нет в United (fuzzy matching по названию)
3. Извлечь авторов, аффилированных с Khazar University
4. Сопоставить департаменты по справочнику
5. Сформировать выходной файл с подсветкой проблемных ячеек

## 1. Конфигурация

**Измените пути к файлам здесь для обработки новых данных**

In [37]:
# ==================== НАСТРОЙКИ ====================

# Пути к файлам
SCOPUS_SOURCE_FILE = '../data/scopus_export_Nov 3-2025_source.xlsx'
UNITED_TARGET_FILE = '../data/United_Scopus and WoS_2024.xlsx'
UNITED_SHEET_NAME = 'Last'  # Название листа в United файле для чтения
DEPARTMENTS_MAPPING_FILE = '../data/departments_mapping.xlsx'
OUTPUT_FILE = 'data/output/new_articles_{year}_{date}.xlsx'  # {date} будет заменен на текущую дату

# Настройки поиска
FUZZY_MATCH_THRESHOLD = 95  # Порог совпадения для fuzzy matching (0-100)
KHAZAR_KEYWORDS = [
    'Khazar University',
    'Khazar',
    'Xəzər Universiteti'
]
# Год(ы) для фильтрации статей
# Варианты:
#   YEAR = 2025              # один год
#   YEAR = [2024, 2025]      # несколько лет
#   YEAR = None              # все годы (без фильтрации)
YEAR = [2025, 2026]

# Фильтрация по Title - исключение статей
# Если Title содержит любую из подстрок (регистр не важен), статья будет исключена
# Пример: ['Correction', 'Erratum', 'Retracted', 'Withdrawn']
TITLE_EXCLUDE_KEYWORDS = ['Correction:', 'Correction to:', 'Erratum to', 'Corrigendum to']

# Настройки форматирования
HIGHLIGHT_COLOR_MULTIPLE_DEPTS = 'FFFF00'  # Желтый для множественных департаментов
HIGHLIGHT_COLOR_NO_DEPT = 'FFFFFF'  # Белый для отсутствующих департаментов

print('✓ Конфигурация загружена')
print(f'  Исходный файл: {SCOPUS_SOURCE_FILE}')
print(f'  Целевой файл: {UNITED_TARGET_FILE}')
print(f'  Лист в United файле: {UNITED_SHEET_NAME}')
print(f'  Справочник департаментов: {DEPARTMENTS_MAPPING_FILE}')
print(f'  Порог fuzzy matching: {FUZZY_MATCH_THRESHOLD}%')
print(f'  Фильтрация по году: {YEAR}')
print(f'  Исключаемые подстроки в Title: {TITLE_EXCLUDE_KEYWORDS if TITLE_EXCLUDE_KEYWORDS else "нет"}')

✓ Конфигурация загружена
  Исходный файл: data/scopus_export_Nov 3-2025_source.xlsx
  Целевой файл: data/United_Scopus and WoS_2024.xlsx
  Лист в United файле: Last
  Справочник департаментов: data/departments_mapping.xlsx
  Порог fuzzy matching: 95%
  Фильтрация по году: [2025, 2026]
  Исключаемые подстроки в Title: ['Correction:', 'Correction to:', 'Erratum to']


## 2. Импорт библиотек

In [38]:
# Основные библиотеки
import pandas as pd
import numpy as np
from datetime import datetime
import re
import os

# Для работы с Excel
import openpyxl
from openpyxl.styles import PatternFill

# Для fuzzy matching
from fuzzywuzzy import fuzz

# Настройки отображения pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 1000)

print('✓ Библиотеки импортированы успешно')
print(f'  pandas: {pd.__version__}')
print(f'  openpyxl: {openpyxl.__version__}')

✓ Библиотеки импортированы успешно
  pandas: 2.3.3
  openpyxl: 3.1.5


## 3. Загрузка данных

In [39]:
print('Загрузка файлов...\n')

# Загрузка исходного файла Scopus
print(f'Загружаем: {SCOPUS_SOURCE_FILE}')
df_scopus = pd.read_excel(SCOPUS_SOURCE_FILE)
print(f'  ✓ Загружено статей: {len(df_scopus)}')
print(f'  ✓ Колонок: {len(df_scopus.columns)}')

# Загрузка целевого файла United
print(f'\nЗагружаем: {UNITED_TARGET_FILE}')
print(f'  Лист: {UNITED_SHEET_NAME}')
df_united = pd.read_excel(UNITED_TARGET_FILE, sheet_name=UNITED_SHEET_NAME)
print(f'  ✓ Загружено статей: {len(df_united)}')
print(f'  ✓ Колонок: {len(df_united.columns)}')

# Загрузка справочника департаментов (если существует)
print(f'\nЗагружаем: {DEPARTMENTS_MAPPING_FILE}')
if os.path.exists(DEPARTMENTS_MAPPING_FILE):
    df_departments = pd.read_excel(DEPARTMENTS_MAPPING_FILE)
    print(f'  ✓ Загружено записей: {len(df_departments)}')
else:
    print('  ⚠ Файл не найден. Будет создан пустой справочник.')
    df_departments = pd.DataFrame(columns=['Author Name', 'Departament'])

print('\n' + '='*70)
print('Просмотр структуры данных:')
print('='*70)
print('\n--- Scopus файл (3 случайные строки Title + Authors) ---')
print(df_scopus[['Title', 'Authors']].sample(3))
print('\n--- United файл (3 случайные строки Title + Authors) ---')
print(df_united[['Title', 'Authors']].sample(3))

Загрузка файлов...

Загружаем: data/scopus_export_Nov 3-2025_source.xlsx
  ✓ Загружено статей: 1334
  ✓ Колонок: 22

Загружаем: data/United_Scopus and WoS_2024.xlsx
  Лист: Last
  ✓ Загружено статей: 1684
  ✓ Колонок: 19

Загружаем: data/departments_mapping.xlsx
  ✓ Загружено записей: 2

Просмотр структуры данных:

--- Scopus файл (3 случайные строки Title + Authors) ---
                                                                                                   Title                                                                                              Authors
613                                Two-body Dirac equation in DSR: Results for fermion-antifermion pairs                                                                              Jafari, N.; Guvendi, A.
600  Numerical investigation on the performance of the solar air collector using jet impingement with...  Afridi, M.I.; Samarmad, A.O.; Ali, M.; Rashid, F.L.; Kadhim, S.A.; Jasim, A.K.; Nayyef, D.R.; Ba...
108     

## 4. Функции обработки

### 4.1 Поиск новых статей (fuzzy matching)

In [40]:
def normalize_title(title):
    """
    Нормализует название статьи для сравнения.
    Убирает лишние пробелы, приводит к нижнему регистру.
    """
    if pd.isna(title):
        return ""
    return re.sub(r'\s+', ' ', str(title).lower().strip())


def find_new_articles(df_source, df_existing, threshold=95):
    """
    Находит статьи из df_source, которых нет в df_existing.
    
    Параметры:
    - df_source: DataFrame с исходными статьями (Scopus)
    - df_existing: DataFrame с существующими статьями (United)
    - threshold: порог fuzzy matching (0-100)
    
    Возвращает:
    - DataFrame с новыми статьями
    - dict с информацией о найденных дубликатах
    """
    print(f'Поиск новых статей (порог fuzzy matching: {threshold}%)...\n')
    
    # Нормализуем названия в существующих статьях
    existing_titles = df_existing['Title'].apply(normalize_title).tolist()
    
    new_articles = []
    duplicates_info = []
    
    for idx, row in df_source.iterrows():
        source_title = normalize_title(row['Title'])
        
        # Проверяем на дубликаты с fuzzy matching
        is_duplicate = False
        best_match_score = 0
        best_match_title = ""
        
        for existing_title in existing_titles:
            similarity = fuzz.ratio(source_title, existing_title)
            
            if similarity > best_match_score:
                best_match_score = similarity
                best_match_title = existing_title

            if similarity >= threshold:
                is_duplicate = True

                duplicates_info.append({
                    'source_title': row['Title'],
                    'matched_title': existing_title,
                    'similarity': similarity
                })
                break
        
        # Если не дубликат, добавляем в список новых
        if not is_duplicate:
            new_articles.append(idx)
        
        # Прогресс
        if (idx + 1) % 100 == 0:
            print(f'  Обработано: {idx + 1}/{len(df_source)} статей')
    
    df_new = df_source.loc[new_articles].copy()
    
    print(f'\n✓ Анализ завершен:')
    print(f'  Всего статей в источнике: {len(df_source)}')
    print(f'  Найдено дубликатов: {len(duplicates_info)}')
    print(f'  Новых статей: {len(df_new)}')
    
    return df_new, duplicates_info


### 4.2 Парсинг авторов из Khazar University

In [41]:
def extract_khazar_authors(authors_with_affiliations, author_full_names, keywords=KHAZAR_KEYWORDS):
    """
    Извлекает авторов из строки с аффилиациями, в именах которых есть слова из `keywords`.
    
    Параметры:
    - authors_with_affiliations: строка "LastName, FirstName, affiliation; LastName, FirstName, affiliation; ..."
    - author_full_names: строка с полными именами и ID "Name (ID); Name (ID); ..."
    - keywords: список ключевых слов для поиска университета
    
    Возвращает:
    - dict с информацией об авторах Khazar:
        {
            'authors_short': 'LastName, F.; LastName2, F.',  # Краткие имена
            'authors_with_ids': 'Full Name (ID); Full Name2 (ID)',  # С ID
            'authors_full': 'Full Name; Full Name2',  # Полные имена
            'count': 2  # Количество авторов
        }
    """
    if pd.isna(authors_with_affiliations):
        return {'authors_short': '', 'authors_with_ids': '', 'authors_full': '', 'count': 0}
    
    # Разбиваем на отдельных авторов (разделитель - точка с запятой)
    author_blocks = str(authors_with_affiliations).split(';')
    
    khazar_authors_short = []
    khazar_authors_with_ids = []
    khazar_authors_full = []
    
    # Парсим author_full_names для получения ID
    full_names_dict = {}
    if not pd.isna(author_full_names):
        # Формат: "LastName, FirstName (ID); LastName, FirstName (ID); ..."
        full_name_parts = str(author_full_names).split(';')
        for part in full_name_parts:
            part = part.strip()
            # Извлекаем имя и ID
            match = re.match(r'(.+?)\s*\((\d+)\)', part)
            if match:
                name = match.group(1).strip()
                author_id = match.group(2).strip()
                # Создаем ключ по фамилии для поиска
                name_parts = name.split(',')
                if len(name_parts) >= 1:
                    last_name = name_parts[0].strip()
                    full_names_dict[last_name] = {'full': name, 'id': author_id}
    
    # Проверяем каждого автора
    for block in author_blocks:
        block = block.strip()
        if not block:
            continue
        
        # Проверяем, содержит ли блок ключевые слова Khazar
        is_khazar = any(keyword.lower() in block.lower() for keyword in keywords)
        
        if is_khazar:
            # Извлекаем имя автора (формат: "LastName, FirstName, affiliation, affiliation, ...")
            # Имя автора - это первые две части до запятых
            parts = block.split(',')
            if len(parts) >= 2:
                last_name = parts[0].strip()
                first_name = parts[1].strip()
                
                # Краткое имя: "LastName, F."
                first_initial = first_name[0] + '.' if first_name else ''
                short_name = f"{last_name}, {first_initial}"
                khazar_authors_short.append(short_name)
                
                # Полное имя с ID из author_full_names
                if last_name in full_names_dict:
                    full_info = full_names_dict[last_name]
                    khazar_authors_with_ids.append(f"{full_info['full']} ({full_info['id']})")
                    khazar_authors_full.append(full_info['full'])
                else:
                    # Если ID не найден, используем имя без ID
                    full_name = f"{last_name}, {first_name}"
                    khazar_authors_with_ids.append(full_name)
                    khazar_authors_full.append(full_name)
    
    return {
        'authors_short': '; '.join(khazar_authors_short),
        'authors_with_ids': '; '.join(khazar_authors_with_ids),
        'authors_full': '; '.join(khazar_authors_full),
        'count': len(khazar_authors_short)
    }

### 4.3 Сопоставление департаментов

In [42]:
def map_departments(authors_string, df_dept_mapping):
    """
    Сопоставляет авторов с департаментами из справочника.
    
    Параметры:
    - authors_string: строка с авторами, разделенными "; " (например, "LastName, F.; LastName2, F.")
    - df_dept_mapping: DataFrame со справочником (колонки: 'Author Name', 'Departament')
    
    Возвращает:
    - dict с информацией:
        {
            'department': строка с департаментом(ами) через "; "
            'needs_highlight': bool - нужна ли желтая подсветка
            'reason': причина подсветки ('not_found', 'multiple', None)
        }
    """
    if pd.isna(authors_string) or not authors_string.strip():
        return {'department': '', 'needs_highlight': False, 'reason': None}
    
    # Разбиваем на отдельных авторов
    authors = [a.strip() for a in str(authors_string).split(';') if a.strip()]
    
    departments = []
    not_found_authors = []
    
    for author in authors:
        # Ищем автора в справочнике (нечувствительно к регистру)
        matches = df_dept_mapping[
            df_dept_mapping['Author Name'].str.lower() == author.lower()
        ]
        
        if len(matches) == 0:
            # Автор не найден
            not_found_authors.append(author)
        else:
            # Автор найден, извлекаем департамент(ы)
            for _, row in matches.iterrows():
                dept = row['Departament']
                if pd.notna(dept) and dept.strip():
                    departments.append(dept.strip())
    
    # Убираем дубликаты департаментов
    departments = list(dict.fromkeys(departments))  # Сохраняет порядок
    
    # Определяем нужна ли подсветка
    needs_highlight = False
    reason = None

    if not_found_authors:
        # Есть авторы без департаментов
        needs_highlight = True
        reason = 'not_found'
    elif len(departments) > 1:
        # Множественные департаменты
        needs_highlight = True
        reason = 'multiple'

    department_str = '; '.join(departments) if departments else ''

    return {
        'department': department_str,
        'needs_highlight': needs_highlight,
        'reason': reason,
        'not_found_authors': not_found_authors
    }


# Тестируем функцию
print('='*70)
print('ТЕСТ: Сопоставление департаментов')
print('='*70 + '\n')

# Создаем тестовый справочник
test_dept_df = pd.DataFrame({
    'Author Name': ['Mahmood, S.', 'Jafari, N.', 'Ali, A.'],
    'Departament': [
        'Aşağı ölçülü materialların tədqiqat mərkəzi',
        'Center for Theoretical Physics',
        'Kimya və kimya mühəndisliyi'
    ]
})

print('Справочник департаментов:')
print(test_dept_df)

print('\n' + '-'*70)

# Тест 1: Автор найден
print('\nТест 1: Один автор найден в справочнике')
result1 = map_departments('Mahmood, S.', test_dept_df)
print(f"Автор: Mahmood, S.")
print(f"Департамент: {result1['department']}")
print(f"Нужна подсветка: {result1['needs_highlight']} ({result1['reason']})")

# Тест 2: Автор не найден
print('\nТест 2: Автор не найден в справочнике')
result2 = map_departments('Unknown, U.', test_dept_df)
print(f"Автор: Unknown, U.")
print(f"Департамент: {result2['department']}")
print(f"Нужна подсветка: {result2['needs_highlight']} ({result2['reason']})")
print(f"Авторы без департамента: {result2['not_found_authors']}")

# Тест 3: Множественные авторы с разными департаментами
print('\nТест 3: Несколько авторов с разными департаментами')
result3 = map_departments('Mahmood, S.; Jafari, N.', test_dept_df)
print(f"Авторы: Mahmood, S.; Jafari, N.")
print(f"Департаменты: {result3['department']}")
print(f"Нужна подсветка: {result3['needs_highlight']} ({result3['reason']})")

# Тест 4: Смешанный случай (один найден, один нет)
print('\nТест 4: Один найден, один нет')
result4 = map_departments('Mahmood, S.; Unknown, U.', test_dept_df)
print(f"Авторы: Mahmood, S.; Unknown, U.")
print(f"Департаменты: {result4['department']}")
print(f"Нужна подсветка: {result4['needs_highlight']} ({result4['reason']})")
print(f"Авторы без департамента: {result4['not_found_authors']}")

ТЕСТ: Сопоставление департаментов

Справочник департаментов:
   Author Name                                  Departament
0  Mahmood, S.  Aşağı ölçülü materialların tədqiqat mərkəzi
1   Jafari, N.               Center for Theoretical Physics
2      Ali, A.                  Kimya və kimya mühəndisliyi

----------------------------------------------------------------------

Тест 1: Один автор найден в справочнике
Автор: Mahmood, S.
Департамент: Aşağı ölçülü materialların tədqiqat mərkəzi
Нужна подсветка: False (None)

Тест 2: Автор не найден в справочнике
Автор: Unknown, U.
Департамент: 
Нужна подсветка: True (not_found)
Авторы без департамента: ['Unknown, U.']

Тест 3: Несколько авторов с разными департаментами
Авторы: Mahmood, S.; Jafari, N.
Департаменты: Aşağı ölçülü materialların tədqiqat mərkəzi; Center for Theoretical Physics
Нужна подсветка: True (multiple)

Тест 4: Один найден, один нет
Авторы: Mahmood, S.; Unknown, U.
Департаменты: Aşağı ölçülü materialların tədqiqat mərkəzi
Нужн

## 5. Основной Pipeline обработки

In [43]:
def process_scopus_data(df_source, df_existing, df_dept_mapping, threshold=95, year=None, title_exclude_keywords=None):
    """
    Основной pipeline обработки данных Scopus.
    
    Параметры:
    - df_source: DataFrame с исходными данными Scopus
    - df_existing: DataFrame с существующими данными United
    - df_dept_mapping: DataFrame со справочником департаментов
    - threshold: порог fuzzy matching для поиска дубликатов
    - year: год(ы) для фильтрации статей
        None - без фильтрации (все годы)
        int - один год (например, 2025)
        list - несколько лет (например, [2024, 2025])
    - title_exclude_keywords: список подстрок для исключения статей по Title (case-insensitive)
    
    Возвращает:
    - df_result: DataFrame с новыми статьями в формате United
    - stats: словарь со статистикой обработки
    """
    
    print('='*70)
    print('ОСНОВНОЙ PIPELINE ОБРАБОТКИ')
    print('='*70 + '\n')

    # Фильтрация по году
    if year is not None:
        # Приводим год к списку для единообразной обработки
        years_list = [year] if isinstance(year, int) else year
        
        print(f'Фильтрация по году: {years_list}')

        # Фильтруем Scopus
        original_scopus_count = len(df_source)
        df_source = df_source[df_source['Year'].isin(years_list)].copy()
        print(f'  Scopus - всего статей: {original_scopus_count}')
        print(f'  Scopus - статей за {years_list}: {len(df_source)}')

        # Фильтруем United
        original_united_count = len(df_existing)
        df_existing = df_existing[df_existing['Year'].isin(years_list)].copy()
        print(f'  United - всего статей: {original_united_count}')
        print(f'  United - статей за {years_list}: {len(df_existing)}')
        print()
    
    # Фильтрация по Title - исключение статей с определенными подстроками
    if title_exclude_keywords and len(title_exclude_keywords) > 0:
        print(f'Фильтрация по Title (исключение статей с подстроками):')
        print(f'  Подстроки для исключения: {title_exclude_keywords}')
        
        before_filter_count = len(df_source)
        
        # Создаем маску: True если Title НЕ содержит ни одной из подстрок
        def should_keep_article(title):
            if pd.isna(title):
                return True
            title_lower = str(title).lower()
            # Если хотя бы одна подстрока найдена - исключаем (return False)
            for keyword in title_exclude_keywords:
                if keyword.lower() in title_lower:
                    print('*'*70+'\n')
                    print(f'keyword: {keyword}')
                    print(f'title_lower: {title_lower}')
                    return False
            return True
        
        df_source = df_source[df_source['Title'].apply(should_keep_article)].copy()
        
        excluded_count = before_filter_count - len(df_source)
        print(f'  Scopus - до фильтрации: {before_filter_count}')
        print(f'  Scopus - исключено статей: {excluded_count}')
        print(f'  Scopus - осталось статей: {len(df_source)}')
        print()

    # Шаг 1: Найти новые статьи (сравниваются только статьи за выбранный год)
    df_new, duplicates = find_new_articles(df_source, df_existing, threshold)
    
    if len(df_new) == 0:
        print('\n⚠ Новых статей не найдено. Обработка завершена.')
        return pd.DataFrame(), {'new_articles': 0, 'khazar_articles': 0, 'highlighted': 0}
    
    # Шаг 2: Обработать каждую новую статью
    print('\n' + '='*70)
    print('Обработка новых статей...')
    print('='*70 + '\n')
    
    result_data = []
    stats = {
        'new_articles': len(df_new),
        'khazar_articles': 0,
        'highlighted_depts': 0,
        'no_khazar_authors': 0
    }
    
    for idx, (_, row) in enumerate(df_new.iterrows(), 1):
        # Извлечь авторов Khazar
        authors_info = extract_khazar_authors(
            row['Authors with affiliations'],
            row['Author full names']
        )
        
        # Пропустить статьи без авторов Khazar
        if authors_info['count'] == 0:
            stats['no_khazar_authors'] += 1
            continue
        
        stats['khazar_articles'] += 1
        
        # Сопоставить департаменты
        dept_info = map_departments(authors_info['authors_short'], df_dept_mapping)
        
        if dept_info['needs_highlight']:
            stats['highlighted_depts'] += 1
        
        # Сформировать запись
        # ВАЖНО: Authors.1 и Author full names содержат ВСЕХ авторов из исходного Scopus файла
        # Authors содержит только авторов Khazar (для согласованности с форматом United)
        result_row = {
            'Departament': dept_info['department'],
            'Authors': authors_info['authors_short'],  # Только Khazar авторы (краткий формат)
            'Authors.1': row['Authors'] if 'Authors' in row else '',  # ВСЕ авторы (краткий формат)
            'Author full names': row['Author full names'] if 'Author full names' in row else '',  # ВСЕ полные имена с ID
            'Title': row['Title'],
            'Year': row['Year'] if 'Year' in row and pd.notna(row['Year']) else '',
            'Source title': row['Source title'] if 'Source title' in row else '',
            'Volume': row['Volume'] if 'Volume' in row else '',
            'Issue': row['Issue'] if 'Issue' in row else '',
            'Art. No.': row['Art. No.'] if 'Art. No.' in row else '',
            'Page start': row['Page start'] if 'Page start' in row else '',
            'Page end': row['Page end'] if 'Page end' in row else '',
            'Page count': row['Page count'] if 'Page count' in row else '',
            'Source': 'Scopus',
            'Təqdimat': '',
            'Data': '',
            'Amount': '',
            'Quartil': '',
            '_highlight': dept_info['needs_highlight'],  # Флаг для подсветки
            '_highlight_reason': dept_info['reason']
        }
        
        result_data.append(result_row)
        
        # Прогресс
        if idx % 10 == 0:
            print(f'  Обработано: {idx}/{len(df_new)} статей')
    
    # Создать итоговый DataFrame
    df_r = pd.DataFrame(result_data)
    
    # Статистика
    print(f'\n✓ Обработка завершена!')
    print(f'\n' + '='*70)
    print('СТАТИСТИКА')
    print('='*70)
    print(f'  Всего новых статей найдено: {stats["new_articles"]}')
    print(f'  Статей с авторами Khazar: {stats["khazar_articles"]}')
    print(f'  Статей без авторов Khazar: {stats["no_khazar_authors"]}')
    print(f'  Статей с подсветкой департаментов: {stats["highlighted_depts"]}')
    print(f'  Итого строк в результате: {len(df_r)}')
    
    return df_r, stats


# Запускаем основной pipeline
print('\n\n')
print('#'*70)
print('# НАЧАЛО ОБРАБОТКИ ДАННЫХ')
print('#'*70 + '\n')

df_result, processing_stats = process_scopus_data(
    df_scopus,
    df_united,
    df_departments,
    threshold=FUZZY_MATCH_THRESHOLD,
    year=YEAR,
    title_exclude_keywords=TITLE_EXCLUDE_KEYWORDS
)

print('\n' + '#'*70)
print('# ОБРАБОТКА ЗАВЕРШЕНА')
print('#'*70)




######################################################################
# НАЧАЛО ОБРАБОТКИ ДАННЫХ
######################################################################

ОСНОВНОЙ PIPELINE ОБРАБОТКИ

Фильтрация по году: [2025, 2026]
  Scopus - всего статей: 1334
  Scopus - статей за [2025, 2026]: 1334
  United - всего статей: 1684
  United - статей за [2025, 2026]: 1271

Фильтрация по Title (исключение статей с подстроками):
  Подстроки для исключения: ['Correction:', 'Correction to:', 'Erratum to']
**********************************************************************

keyword: Correction:
title_lower: correction: ai-supported spherical fuzzy decision-making for barriers to renewable energy projects in hospitals: comparative country analysis (energy informatics, (2025), 8, 1, (120), 10.1186/s42162-025-00577-7)
**********************************************************************

keyword: Correction:
title_lower: publisher correction: assessing carbon–neutral supercapacitors in rene

## 6. Экспорт в Excel с форматированием

In [44]:
def export_to_excel_with_highlighting(df, output_path, highlight_color='FFFF00'):
    """
    Экспортирует DataFrame в Excel с желтой подсветкой проблемных ячеек.
    
    Параметры:
    - df: DataFrame с данными (должен содержать колонки _highlight и _highlight_reason)
    - output_path: путь к выходному файлу
    - highlight_color: цвет подсветки в формате RGB hex (по умолчанию желтый)
    """
    
    if len(df) == 0:
        print('⚠ DataFrame пустой, нечего экспортировать.')
        return
    
    # Создаем копию без служебных колонок
    df_export = df.drop(columns=['_highlight', '_highlight_reason'], errors='ignore').copy()
    
    # Экспортируем в Excel
    print(f'Экспорт в файл: {output_path}')
    df_export.to_excel(output_path, index=False, engine='openpyxl')
    print(f'  ✓ Данные записаны ({len(df_export)} строк)')
    
    # Открываем файл для форматирования
    wb = openpyxl.load_workbook(output_path)
    ws = wb.active
    
    # Определяем желтый цвет для подсветки
    yellow_fill = PatternFill(start_color=highlight_color, end_color=highlight_color, fill_type='solid')
    
    # Применяем подсветку
    highlighted_count = 0
    for idx, row in df.iterrows():
        if row.get('_highlight', False):
            # Индекс строки в Excel (учитываем заголовок)
            excel_row_idx = idx + 2  # +1 для заголовка, +1 для индексации с 0
            
            # Подсвечиваем ячейку в колонке Departament (колонка A, индекс 1)
            cell = ws.cell(row=excel_row_idx, column=1)
            cell.fill = yellow_fill
            highlighted_count += 1
    
    # Сохраняем
    wb.save(output_path)
    print(f'  ✓ Применено форматирование ({highlighted_count} ячеек подсвечено)')
    print(f'\n✓ Файл успешно создан: {output_path}')


# Проверяем, есть ли результаты для экспорта
if len(df_result) > 0:
    # Формируем имя файла с текущей датой и годом/годами
    current_date = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
    
    # Формируем строку года для имени файла
    if YEAR is None:
        year_str = 'all_years'
    elif isinstance(YEAR, int):
        year_str = str(YEAR)
    else:  # список годов
        year_str = '-'.join(map(str, sorted(YEAR)))
    
    output_filename = OUTPUT_FILE.replace('{year}', year_str).replace('{date}', current_date)
    
    print('='*70)
    print('ЭКСПОРТ РЕЗУЛЬТАТОВ В EXCEL')
    print('='*70 + '\n')
    
    # Экспортируем
    export_to_excel_with_highlighting(
        df_result,
        output_filename,
        highlight_color=HIGHLIGHT_COLOR_MULTIPLE_DEPTS
    )
    
    # Дополнительная информация
    print(f'\n' + '='*70)
    print('ИНФОРМАЦИЯ О ПОДСВЕЧЕННЫХ ЯЧЕЙКАХ')
    print('='*70)
    
    # Подсчет причин подсветки
    highlight_reasons = df_result[df_result['_highlight'] == True]['_highlight_reason'].value_counts()
    print(f'\nПричины подсветки:')
    for reason, count in highlight_reasons.items():
        reason_text = {
            'not_found': 'Автор не найден в справочнике',
            'multiple': 'Множественные департаменты'
        }.get(reason, reason)
        print(f'  {reason_text}: {count} статей')
    
    print(f'\n📄 Откройте файл для просмотра:')
    print(f'   {output_filename}')
    print(f'\nЖелтая подсветка означает, что нужно вручную проверить департамент.')
    
else:
    print('⚠ Нет результатов для экспорта (df_result пустой)')

ЭКСПОРТ РЕЗУЛЬТАТОВ В EXCEL

Экспорт в файл: data/output/new_articles_2025-2026_2025-11-04-16-39-46.xlsx
  ✓ Данные записаны (67 строк)
  ✓ Применено форматирование (67 ячеек подсвечено)

✓ Файл успешно создан: data/output/new_articles_2025-2026_2025-11-04-16-39-46.xlsx

ИНФОРМАЦИЯ О ПОДСВЕЧЕННЫХ ЯЧЕЙКАХ

Причины подсветки:
  Автор не найден в справочнике: 67 статей

📄 Откройте файл для просмотра:
   data/output/new_articles_2025-2026_2025-11-04-16-39-46.xlsx

Желтая подсветка означает, что нужно вручную проверить департамент.
